<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/DZ_RNN2_v3_%D1%80%D0%B0%D0%B1%D0%BE%D1%87%D0%B0%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
'''Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:
x - последовательность цифр
y(0) = x(0), y(i) = x(i) + x(0). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU'''

'Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)\nи задавались следующим образом:\nx - последовательность цифр\ny(0) = x(0), y(i) = x(i) + x(0). Если y(i) >= 10, то y(i) = y(i) - 10\n\nЗадача:\nнаучить модель предсказывать y(i) по x(i)\nпробовать RNN, LSTM, GRU'

In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split

In [102]:
x=[]
y=[]
y_sum=[]
max_str=10000

In [103]:
X_1 = torch.randint(0,9,(max_str,32))
x_np = X_1.numpy()

for i in range(max_str):
    x=x_np[i].tolist()

    y_vr=0
    y.append(x[0])
    for i in range(1,len(x)):
        y_vr = x[i]+x[0]
        if (y_vr < 10):
            y.append(y_vr)
        else:
            y.append(y_vr-10)

    y_sum.append(y)
    y=[]
tar=torch.from_numpy(np.array(y_sum))

In [104]:
X_1

tensor([[8, 1, 3,  ..., 5, 6, 0],
        [8, 0, 1,  ..., 3, 2, 4],
        [4, 8, 5,  ..., 7, 2, 1],
        ...,
        [7, 4, 4,  ..., 3, 5, 2],
        [3, 5, 5,  ..., 3, 8, 0],
        [5, 8, 1,  ..., 0, 5, 3]])

In [105]:
tar

tensor([[8, 9, 1,  ..., 3, 4, 8],
        [8, 8, 9,  ..., 1, 0, 2],
        [4, 2, 9,  ..., 1, 6, 5],
        ...,
        [7, 1, 1,  ..., 0, 2, 9],
        [3, 8, 8,  ..., 6, 1, 3],
        [5, 3, 6,  ..., 5, 0, 8]])

In [106]:
# Преобразование размерностей для RNN
X = X_1.unsqueeze(-1)            # (10000, 32, 1) - добавляем размерность признака
y = tar.unsqueeze(-1)            # (10000, 32, 1)

In [107]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X.numpy(), y.numpy(), test_size=0.2, random_state=42
)

In [108]:
# Конвертация в тензоры PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [109]:
# Создание Dataset и DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [110]:
# Определение модели RNN
class RNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True  # Формат входа: (batch, seq, features)
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        output = self.fc(gru_out)
        return output

In [111]:
# Инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNModel().to(device)
print(model)

RNNModel(
  (gru): GRU(1, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)


In [112]:
# Функция потерь и оптимизатор
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [113]:
# Обучение модели
num_epochs = 20

for epoch in range(num_epochs):
    # Режим обучения
    model.train()
    train_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Прямой проход
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Обратный проход и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * batch_X.size(0)


In [114]:
# Валидация
model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item() * batch_X.size(0)

test_loss = test_loss / len(test_loader.dataset)

print(f'Epoch [{epoch+1}/{num_epochs}] | '
          f'Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}')



Epoch [20/20] | Train Loss: 1907.631564 | Test Loss: 0.203897


In [120]:
# Вычисление MSE на всем тестовом наборе
model.eval()
with torch.no_grad():

    test_preds = model(X_test.to(device))
    final_loss = criterion(test_preds, y_test.to(device))
    print(f'\nFinal Test MSE: {final_loss.item():.6f}')




Final Test MSE: 0.203897


In [134]:
# Предсказания
sample = X_test[:1].to(device)
predictions = model(sample)
X_1=X_test[:1].numpy() #Исходная последовательность
predict_1=np.round(predictions.detach().numpy()) #предсказанная последовательность

result = np.c_[X_1,predict_1]
print(result)


[[[6. 6.]
  [2. 8.]
  [8. 4.]
  [7. 3.]
  [7. 3.]
  [0. 6.]
  [4. 1.]
  [5. 1.]
  [1. 7.]
  [8. 4.]
  [3. 8.]
  [4. 1.]
  [5. 1.]
  [1. 7.]
  [5. 1.]
  [7. 3.]
  [0. 6.]
  [8. 4.]
  [0. 6.]
  [4. 1.]
  [4. 1.]
  [1. 7.]
  [7. 3.]
  [4. 1.]
  [3. 8.]
  [4. 1.]
  [2. 8.]
  [5. 1.]
  [8. 4.]
  [8. 4.]
  [6. 2.]
  [7. 3.]]]
